In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
# 多行输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [3]:
from fastai import *
from fastai.text import *

In [7]:
doc(Config)

class Config [source] [test] Config () 
 
 × Tests found for Config : pytest -sv tests/test_datasets.py::test_creates_config [source] pytest -sv tests/test_datasets.py::test_default_config [source] pytest -sv tests/test_datasets.py::test_load_config [source] pytest -sv tests/test_datasets.py::test_user_config [source] To run tests please refer to this guide . Creates a default config file 'config.yml' in $FASTAI_HOME (default ~/.fastai/ ) 
 Show in docs

- IMDB 精简数据

In [28]:
path = untar_data(URLs.IMDB)
path
path.ls()

PosixPath('/home/lab/.fastai/data/imdb')

[PosixPath('/home/lab/.fastai/data/imdb/tmp_lm'),
 PosixPath('/home/lab/.fastai/data/imdb/test'),
 PosixPath('/home/lab/.fastai/data/imdb/tmp_clas'),
 PosixPath('/home/lab/.fastai/data/imdb/unsup'),
 PosixPath('/home/lab/.fastai/data/imdb/train'),
 PosixPath('/home/lab/.fastai/data/imdb/imdb.vocab'),
 PosixPath('/home/lab/.fastai/data/imdb/models'),
 PosixPath('/home/lab/.fastai/data/imdb/imdb_textlist_class'),
 PosixPath('/home/lab/.fastai/data/imdb/data_lm'),
 PosixPath('/home/lab/.fastai/data/imdb/README')]

In [29]:
BATCH = 32

In [30]:
# data_lm = (TextList.from_folder(path)
#            #Inputs: all the text files in path
#             .filter_by_folder(include=['train', 'test']) 
#            #We may have other temp folders that contain text files so we only keep what's in train and test
#             .split_by_rand_pct(0.1)
#            #We randomly split and keep 10% (10,000 reviews) for validation
#             .label_for_lm()           
#            #We want to do a language model so we label accordingly, 自己就是自己的标签
#             .databunch(bs=BATCH))

In [31]:
# data_lm.save('data_lm')

In [32]:
data_lm = load_data(path, 'data_lm', bs=BATCH)

## 创建模型

In [33]:
learn_lm = language_model_learner(data_lm, Transformer, drop_mult=0.3)

In [34]:
learn_lm.loss_func
learn_lm.opt_func

FlattenedLoss of CrossEntropyLoss()

functools.partial(<class 'torch.optim.adam.Adam'>, betas=(0.9, 0.99))

In [35]:
learn_lm.model

SequentialRNN(
  (0): Transformer(
    (encoder): Embedding(49852, 768)
    (pos_enc): Embedding(512, 768)
    (drop_emb): Dropout(p=0.03)
    (layers): ModuleList(
      (0): DecoderLayer(
        (mhra): MultiHeadAttention(
          (attention): Linear(in_features=768, out_features=2304, bias=True)
          (out): Linear(in_features=768, out_features=768, bias=True)
          (drop_att): Dropout(p=0.03)
          (drop_res): Dropout(p=0.03)
          (ln): LayerNorm(torch.Size([768]), eps=1e-05, elementwise_affine=True)
        )
        (ff): SequentialEx(
          (layers): ModuleList(
            (0): Linear(in_features=768, out_features=3072, bias=True)
            (1): GeLU()
            (2): Linear(in_features=3072, out_features=768, bias=True)
            (3): Dropout(p=0.03)
            (4): MergeLayer()
            (5): LayerNorm(torch.Size([768]), eps=1e-05, elementwise_affine=True)
          )
        )
      )
      (1): DecoderLayer(
        (mhra): MultiHeadAttention

### 编码器

In [36]:
encoder = get_model(learn_lm.model)[0]
encoder

Transformer(
  (encoder): Embedding(49852, 768)
  (pos_enc): Embedding(512, 768)
  (drop_emb): Dropout(p=0.03)
  (layers): ModuleList(
    (0): DecoderLayer(
      (mhra): MultiHeadAttention(
        (attention): Linear(in_features=768, out_features=2304, bias=True)
        (out): Linear(in_features=768, out_features=768, bias=True)
        (drop_att): Dropout(p=0.03)
        (drop_res): Dropout(p=0.03)
        (ln): LayerNorm(torch.Size([768]), eps=1e-05, elementwise_affine=True)
      )
      (ff): SequentialEx(
        (layers): ModuleList(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GeLU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
          (3): Dropout(p=0.03)
          (4): MergeLayer()
          (5): LayerNorm(torch.Size([768]), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (1): DecoderLayer(
      (mhra): MultiHeadAttention(
        (attention): Linear(in_features=768, out_features=2304, bias=T

- 生成假的评论，使用语言模型预测

In [37]:
TEXT = "The color of the sky is"
N_WORDS = 40
N_SENTENCES = 2

In [38]:
print("\n\n".join(learn_lm.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

The color of the sky is a t n coast pig , f time ire t o and bed mas t o f have , o f time n ached t o f two ch den her man n house , her such that ks ,

The color of the sky is f time had f ous her man de looked n friend , o f have , o f time de ted , o f fair that circle t o f night inside es asks , o f me n lobby


- temperature 控制生成文本的随机性

In [39]:
print("\n\n".join(learn_lm.predict(TEXT, N_WORDS, temperature=0.1) for _ in range(N_SENTENCES)))

The color of the sky is his t o f time de n house , o f time de n house , o f time de n breath , o f time de , o f time de n house , o f time de ,

The color of the sky is his t o f time de n house , o f time de n times , o f time de n house , o f time de n first , o f time de n times , o f time


- 预训练数据集

In [40]:
tr_itos = pickle.load(open(Config().model_path()/'transformer/itos_tfmer.pkl', 'rb'))

In [41]:
tr_itos[:10]

['.</c>',
 ',</c>',
 't</c>',
 'h</c>',
 'e</c>',
 '"</c>',
 'o</c>',
 'a</c>',
 'n</c>',
 'd</c>']

In [42]:
len(tr_itos), len(data_lm.vocab.itos) # wiki 词汇， IMDB 词汇

(40478, 49852)

In [43]:
unks[:16]

[(0, 'xxunk'),
 (1, 'xxpad'),
 (2, 'xxbos'),
 (3, 'xxeos'),
 (4, 'xxfld'),
 (5, 'xxmaj'),
 (6, 'xxup'),
 (7, 'xxrep'),
 (8, 'xxwrep')]

## 精调模型

In [44]:
learn_lm.lr_find()

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


RuntimeError: CUDA out of memory. Tried to allocate 426.00 MiB (GPU 0; 7.93 GiB total capacity; 6.04 GiB already allocated; 370.44 MiB free; 416.40 MiB cached)

In [ ]:
learn_lm.recorder.plot(skip_end=10)

In [28]:
learn_lm.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,5.690483,6.580764,0.042857,36:24


In [ ]:
learn_lm.save('transformer_fit_1')

In [37]:
learn_lm=None
gc.collect()

0

- 保存

In [38]:
data_lm = load_data(path, 'data_lm', bs=BATCH//2)

In [39]:
learn_lm = language_model_learner(data_lm, Transformer, drop_mult=0.3)

In [40]:
learn_lm.unfreeze()

In [41]:
learn_lm.load('transformer_fit_1');

In [42]:
learn_lm.fit_one_cycle(1, 1e-3, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,6.425639,6.407260,0.083550,55:43


In [43]:
learn_lm.save('transformer_fit_2')

In [44]:
learn_lm=None
gc.collect()

11

In [45]:
learn_lm = language_model_learner(data_lm, Transformer, drop_mult=0.3)

In [46]:
learn_lm.load('transformer_fit_2');

In [47]:
TEXT = "i liked this movie because"
N_WORDS = 40
N_SENTENCES = 2

In [48]:
print("\n\n".join(learn_lm.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

i liked this movie because a " Location going Is the to the It it , parts and in if he . the and in Of as finally , . 's a , did i to the us the a

i liked this movie because to to Of and Is 's . the and . , the . ( the the " , And Song to and the you to ? due in if the


In [49]:
print("\n\n".join(learn_lm.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

i liked this movie because there of genre know times ) To , her . , , , THIS time Is for to ( . are there The fact one bad for the the the , the To

i liked this movie because . in would ! to one was To , but The - The , the When the Involves The of . seem and in the the . movie


In [50]:
learn_lm.save_encoder('fine_tuned_enc')

In [51]:
learn_lm = None
gc.collect()

0

## 语义分类任务

In [13]:
learn_c = None
gc.collect()

0

In [1]:
path = untar_data(URLs.IMDB)
BATCH = 16

NameError: name 'untar_data' is not defined

In [15]:
# data_clas = (TextList.from_folder(path, vocab=data_lm.vocab)
#              #grab all the text files in path
#              .split_by_folder(valid='test')
#              #split by train and valid folder (that only keeps 'train' and 'test' so no need to filter)
#              .label_from_folder(classes=['neg', 'pos'])
#              #label them all with their folders
#              .databunch(bs=BATCH))

In [16]:
# data_clas.save('imdb_textlist_class')

In [17]:
data_clas = load_data(path, 'imdb_textlist_class', bs=BATCH//4)

In [18]:
data_clas.show_batch()

text,target
xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules,pos
"xxbos xxmaj warning : xxmaj spoilers xxmaj galore ! \n \n xxmaj tim xxmaj burton remaking this sui generis movie is about as sensible as remaking xxmaj psycho - oh , that 's right , some idiot already did that - i rest my case . \n \n xxmaj movie opens with xxunk blundering a simulation , proving he 's not that smart from the outset . xxmaj",neg
"xxbos xxmaj already his first claim , that desires are always artificial , is totally fallacious . \n \n xxmaj when a xxmaj jehovah xxmaj witness reject gets his own documentary on movies  or anything for that matter - it 's time for anyone to get their own . xxmaj although far , far more intelligent than , say , xxmaj paris xxmaj hilton ( i know ,",neg
"xxbos i saw xxmaj heartland when it was first released in 1980 and i have just seen it again . xxmaj it improves with age . xxmaj heartland is not just for lovers of "" indie "" films . xxmaj at a time when most xxmaj american films are little more than cynical attempts to make money with xxup cgi , pyrotechnics , and / or vulgarity , xxmaj heartland",pos


In [19]:
learn_c = text_classifier_learner(data_clas, Transformer, drop_mult=0.5)

RuntimeError: CUDA out of memory. Tried to allocate 146.12 MiB (GPU 0; 7.93 GiB total capacity; 6.22 GiB already allocated; 10.44 MiB free; 29.63 MiB cached)

In [ ]:
learn_c.model

In [10]:
learn_c.loss_func

FlattenedLoss of CrossEntropyLoss()

In [11]:
learn_c.load_encoder('fine_tuned_enc')

In [12]:
learn_c.lr_find()
learn_c.recorder.plot()

RuntimeError: CUDA out of memory. Tried to allocate 146.12 MiB (GPU 0; 7.93 GiB total capacity; 6.22 GiB already allocated; 10.44 MiB free; 29.63 MiB cached)

In [16]:
learn_c.fit_one_cycle(3, 5e-3, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time


RuntimeError: CUDA error: device-side assert triggered

In [15]:
learn_c.save('fine1')

RuntimeError: cuda runtime error (59) : device-side assert triggered at /opt/conda/conda-bld/pytorch_1549630534704/work/torch/csrc/generic/serialization.cpp:23

In [ ]:
learn_c = None
gc.collect()

NameError: name 'Config' is not defined